In [1]:
#Import Dependencies 
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report












In [3]:
#Import and read the clean csv data 
urgentcare_df=pd.read_csv("/urgentcare_clean.csv")
urgentcare_df.head()




,diagnosis_category,diagnosis_sub_category,treatment_category,treatment_sub_category,determination,type,age_range,patient_gender
0,Infectious,Hepatitis,Pharmacy/Prescription Drugs,Anti-virals,Overturned Decision of Health Plan,Medical Necessity,41-50,Male
1,Mental,Eating Disorder,Mental Health Treatment,Residential Treatment Center - Admission,Upheld Decision of Health Plan,Medical Necessity,21-30,Female
2,Autism Spectrum,Autism-PDD-NOS,Autism Related Treatment,Speech Therapy,Upheld Decision of Health Plan,Medical Necessity,0-10,Female
3,Prevention/Good Health,NaN,"Diagnostic Imaging, Screening and Testing",Mammography,Overturned Decision of Health Plan,Experimental/Investigational,65+,Female
4,Prevention/Good Health,NaN,"Diagnostic Imaging, Screening and Testing",Lab Work,Upheld Decision of Health Plan,Experimental/Investigational,21-30,Male


In [4]:
#Determine the number of unique values in each column
urgentcare_df.nunique()

diagnosis_category         27
diagnosis_sub_category    145
treatment_category         30
treatment_sub_category    120
determination               2
type                        3
age_range                   7
patient_gender              2
dtype: int64

In [5]:
#Look at type value counts 
type_count= urgentcare_df["type"].value_counts()
type_count

Medical Necessity               821
Experimental/Investigational    468
Urgent Care                      18
Name: type, dtype: int64

In [10]:
urgentcare_df['target']=np.where(urgentcare_df["type"]=="Medical Necessity", "Medically Necessary", "Experimental/Investigation or Nonemergent")
urgentcare_df['target'].value_counts()



Medically Necessary                          821
Experimental/Investigation or Nonemergent    486
Name: target, dtype: int64

In [11]:
urgentcare_df['target']=np.where(urgentcare_df["target"]=="Medically Necessary", 1, 0)
urgentcare_df['target'].value_counts()

1    821
0    486
Name: target, dtype: int64

In [12]:
urgentcare_df=urgentcare_df.drop('type', axis = 1)

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df =pd.get_dummies(urgentcare_df)
dummies_df.head()

,target,diagnosis_category_Autism Spectrum,diagnosis_category_Blood Related,diagnosis_category_Cancer,diagnosis_category_Cardiac/Circulatory,diagnosis_category_Central Nervous System/ Neuromuscular,diagnosis_category_Chronic Pain,diagnosis_category_Dental,diagnosis_category_Digestive System/ Gastrointestinal,"diagnosis_category_Ears, Nose, Throat",...,determination_Upheld Decision of Health Plan,age_range_0-10,age_range_11_20,age_range_21-30,age_range_31-40,age_range_41-50,age_range_51-64,age_range_65+,patient_gender_Female,patient_gender_Male
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
2,1,1,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1


In [14]:
# Split our preprocessed data into our features and target arrays
y=dummies_df["target"].values 
X=dummies_df.drop('target', axis = 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state= 42)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Testing a deep neural network 

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layer = len(X_train_scaled[0])
hidden_layer_1= 350
hidden_layer_2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_1, activation="relu", input_dim = input_layer))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer_2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()


# Compile the model
nn.compile(loss="binary_crossentropy", optimizer= "adam", metrics=["accuracy"])


# Train the model
fit_model = nn.fit (X_train_scaled, y_train, epochs= 100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 350)               116900    
                                                                 
 dense_1 (Dense)             (None, 30)                10530     
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 127,461
Trainable params: 127,461
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
28/28 [==============================] - 1s 7ms/step - loss: 0.4808 - accuracy: 0.7817
Epoch 2/100
28/28 [==============================] - 0s 8ms/step - loss: 0.1831 - accuracy: 0.9349
Epoch 3/100
28/28 [==============================] - 0s 5ms/step - loss: 0.1158 - accuracy: 0.9600
Epoch 4/100
28/28

# Testing Logistic Regression

In [17]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train, y_train)

In [18]:
# Make a prediction using the testing data
testing_predictions = logistic_regression_model.predict(X_test)

testing_predictions

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,

In [19]:
# Print the balanced_accuracy score of the model
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, testing_predictions)

0.8558707045794105

In [20]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix


training_matrix = confusion_matrix(y_test, testing_predictions)


print(training_matrix)

[[115  34]
 [ 17 266]]


In [21]:
# Print the classification report for the model
testing_report = classification_report(y_test, testing_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       0.87      0.77      0.82       149
           1       0.89      0.94      0.91       283

    accuracy                           0.88       432
   macro avg       0.88      0.86      0.87       432
weighted avg       0.88      0.88      0.88       432



# Testing Logistic Regression after Over Sampling the Data 

In [22]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
oversample = RandomOverSampler(sampling_strategy='minority',random_state=1)

# Fit the original training data to the random_oversampler model
X_over, y_over = oversample.fit_resample(X_train, y_train)

In [23]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
lr_model = logistic_regression_model.fit(X_over, y_over)

# Make a prediction using the testing data
testing_predictions = logistic_regression_model.predict(X_test)

testing_predictions

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,

In [24]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test, testing_predictions)

0.8468589181113193

In [25]:
# Generate a confusion matrix for the model
training_matrix = confusion_matrix(y_test, testing_predictions)


print(training_matrix)

[[116  33]
 [ 24 259]]


In [26]:
# Print the classification report for the model
testing_report = classification_report(y_test, testing_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       0.83      0.78      0.80       149
           1       0.89      0.92      0.90       283

    accuracy                           0.87       432
   macro avg       0.86      0.85      0.85       432
weighted avg       0.87      0.87      0.87       432

